In [14]:
import cv2
import math
import numpy as np

#
# モード設定変数(共通)
#
USE_LINE_INSECT = 3 # [昆虫画像256/line切り替え設定] 2: AIの線画 0: 256昆虫画像を使用する 1: line昆虫画像を使用する
DISP_DOG_WINDOW = 0 # [犬画像の表示設定] 0:最終画像以外の表示なし 1: 初期ロード直後の画像も表示 2: さらに、輪郭抽出時の加工画像等も表示
BUNKATSU = 4 #中間層の分割数

#
# モード設定変数(輪郭配置関連)
#
PLACE_EDGE_INSECTS = 1 # 0:輪郭線沿いに昆虫を表示しない 1:輪郭線沿いに昆虫を表示する
DRAW_EDGE_FIGURE = 0 # 1:昆虫画像位置ガイドとしての直線・楕円を描画する  0:昆虫画像位置ガイドとしての直線・楕円を描画しない
DISP_EDGE_INSECT_WINDOW = 0 # [別ウィンドウでの昆虫の表示] 0:表示なし 1: アフィン変換後の画像を表示 2: さらに、初期ロード直後の画像も表示

RED = (0, 0, 255)
GREEN = (0, 255, 0)
DARKGREEN = (48, 232, 48)
CYAN = (255, 192, 0)
BLUE = (255, 0, 0)
BROWN = (42, 42, 165)
BLACK = (0, 0, 0)
YELLOW = (0, 255, 255)

THCK = 1
THCK2 = 2

# 昆虫名称群(全て)
insect_images_all = [
    'beetle','beetle2', 'gomimusi','gomimusi2','gomimusi3', 'kamemusi','kamemusi2','kamemusi3', 
    'kogane','kogane2','kogane3', 'maimai','maimai2','maimai3', 'tobiiro', 'vespa','vespa2','vespa3',
    'yokobai','yokobai2','yokobai3',
]
#################################################
insect_images_all_2 = [
    'beetle','beetle2', 'gomimusi', 'kamemusi','kamemusi2','kamemusi3', 
    'kogane','kogane2','kogane3', 'maimai','maimai2','maimai3', 'tobiiro', 'vespa','vespa2','vespa3',
    'yokobai','yokobai2','yokobai3',
]

# 昆虫名称群(非側面画像)
insect_images = [
    'beetle', 'gomimusi', 'kamemusi', 'kogane', 'maimai', 'tobiiro', 'vespa', 'yokobai',
]
#########################################
insect_images_AI = [
    'beetle', 'kamemusi', 'maimai', 'mantis','vespa','snail', 'weevil', 'locust',
]
insect_images_AI_new = [
    'hornet6', 'grasshopper3', 'leaf3', 'grasshopper4','cicada1','mantis4', 'cicada6',
    'butterfly12', 'butterfly13', 'hornet1', 'mantis','snail',
    'hornet8','butterfly16','grasshopper9','mantis6','beetle1',
    'pillbug1',
]
insect_images_AI_new_p = [
    'stinkbug1','tamamusi2','butterfly16','butterfly18',
    'hornet6', 'grasshopper3', 'leaf3', 'grasshopper4','cicada6','mantis4', 
    'butterfly17','hornet1', 'mantis',
    'hornet8','mantis6','beetle1','pillbug1',
    
]
insect_images_AI_around = [
    'hornet6', 'leaf3',  
    'butterfly12', 'butterfly13', 'hornet1', 'mantis','snail', 'weevil','hornet8',
]
insect_images_AI_c = [
    'beetle', 'vespa', 'maimai', 'mantis','kamemusi','snail', 'weevil', 'locust',
]
insect_images_AI_c_new = [
    'mantis8',#'mantis5-2','mantis6-2',#'mantis7','mantis5','mantis4',
]

####################################################

# 昆虫画像によりminAreaRectでの矩形抽出状況(広い/狭い)に差があるため、補正用係数を設定する
coef_adj_scale = [
    1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 
    0.75, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 
    1.0, 1.0, 1.0,
]

# 昆虫画像によりmask/line画像と256画像で回転角に差があるため、補正用係数を設定する
additional_angle = [
    0, 270, 0, 270, 270, 0, 270, 270, 
    0, 90, 270, 0, 270, 270, 0, 0, 270, 270,
    0, 270, 270,
]
###############################################
additional_angle_AI = [
    0, 270, 0, 240, 0, 0, 0, 0,
]
additional_angle_AI_new = [
    0, 180, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
    0, 0, 0, 0, 0, 0, 0, 0,
]
additional_angle_AI_around = [
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
    0, 0, 0,
]
additional_angle_AI_middle = [
    10, 0, 90, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
]

# ミラーリング処理が必要な昆虫画像があるため、フラグ保持する
flag_mirror = [
    0, 0, 0, 0, 0, 0, 0, 0, 
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
    0, 1, 1,
]
flag_mirror_AI = [
    0, 0, 0, 1, 0, 0, 0, 0,
]
flag_mirror_AI_new = [
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
    
]
flag_mirror_AI_around = [
    0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
    0, 0, 0,
]

###############################################

# -----------------------------------------------------
# 画像表示関数
# -----------------------------------------------------
def disp_img(title, a_img):
    cv2.imshow(title, a_img)
    cv2.waitKey(0)

# -----------------------------------------------------
# 中点座標取得関数
# -----------------------------------------------------
def calc_midpoint(start_p, end_p):
    return (start_p[0] + end_p[0]) // 2, (start_p[1] + end_p[1]) // 2

# -----------------------------------------------------
# 直線上の任意点座標を取得
# -----------------------------------------------------
def get_point_on_line(point_start, point_end, offset):
    point_x_start = point_start[0]
    point_y_start = point_start[1]
    point_x_end = point_end[0]
    point_y_end = point_end[1]

    adj_x = 0

    diff_x = abs(point_x_start - point_x_end)
    ofs_x = int(diff_x * offset)
    if (point_x_start < point_x_end):
        adj_x = point_x_start + ofs_x
    else:
        adj_x = point_x_start - ofs_x

    adj_y = 0

    diff_y = abs(point_y_start - point_y_end)
    ofs_y = int(diff_y * offset)
    if (point_y_start < point_y_end):
        adj_y = point_y_start + ofs_y
    else:
        adj_y = point_y_start - ofs_y

    return ([adj_x, adj_y])

# -----------------------------------------------------
# 距離取得関数
# -----------------------------------------------------
def calc_distance(start_p, end_p):
    delta_x, delta_y = calc_delta(start_p, end_p)
    return int(math.sqrt(delta_x**2 + delta_y**2))

# -----------------------------------------------------
# 回転角取得関数
# -----------------------------------------------------
# delta_x, delta_y = [0, 0]

def calc_angle(start_p, end_p):
    delta_x, delta_y = calc_delta(start_p, end_p)

    angle_radians = math.atan2(delta_y, delta_x)
    angle_degrees = math.degrees(angle_radians)

#    if angle_degrees < 0:
#        angle_degrees += 360

    return (angle_degrees + 90) % 360

# -----------------------------------------------------
# デルタ取得関数
# -----------------------------------------------------
def calc_delta(start_p, end_p):
    delta_x = end_p[0] - start_p[0]
    delta_y = end_p[1] - start_p[1]

    return delta_x, delta_y

# ------------------------------------------------------------------------------------------------------
# 画像を受け取り、ロード、グレースケール化、二値化、輪郭抽出、最大輪郭判定を行い、最大輪郭を返却する
# ------------------------------------------------------------------------------------------------------
def get_largest_contour(img, disp_image):

    # 画像をグレースケールに変換
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    if disp_image != 0:
        disp_img('img_gray', img_gray)

    # 画像を二値化
    ret, img_bin = cv2.threshold(img_gray, 127, 255,0)
    if disp_image != 0:
        disp_img('img_bin', img_bin)

    # 画像輪郭抽出
    contours, hierarchy = cv2.findContours(img_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    #   
    # print('num of contours=' + str(len(contours)))

    # 最大輪郭決定
    largest_contour = []
    num_points = 0
    for cnt in contours:
        if len(cnt) > num_points:
            num_points = len(cnt)
            largest_contour = cnt

    return largest_contour

# -------------------------------------------------------------------------------------------------------------------------
# 4 昆虫画像と背景画像との合成処理 (共通関数)
#     img_insect画像の中心点と、img_backの引数指定された中心点が合うように位置合わせを行い合成する
#     背景の犬画像に対し外側にはみ出る部分がある場合は、その部分をカットして合成する(Lengthと開始点xまたはyを調整)
# -------------------------------------------------------------------------------------------------------------------------
def process_image(img_insect, img_back, midpoint_x, midpoint_y):

    insect_height = img_insect.shape[0]
    insect_width = img_insect.shape[1]

    backimg_height = img_back.shape[0]
    backimg_width = img_back.shape[1]

    # (1) 背景画像指定点(楕円またはパーツ中心点)と、昆虫画像の中心点が一致する形となるよう配置位置を決定する
    # (1)-1
    ###################################################半分
    y_reduce_upper = 0
    y_offset = midpoint_y - insect_height // 2
    if (y_offset < 0):
        y_reduce_upper = -y_offset
        y_offset = 0

    # (1)-2
    x_reduce_left = 0
    x_offset = midpoint_x - insect_width // 2
    ###################################################半分
    if (x_offset < 0):
        x_reduce_left = -x_offset
        x_offset = 0

    # (2) 背景画像合成位置(左上、右下座標)の決定
    # (1-1)あるいは(1-2)において、昆虫画像位置がマイナスとなり0に補正した場合は、その分、右側、あるいは下側の座標値を減らし調整する
    y1, y2 = y_offset, y_offset + insect_height - y_reduce_upper
    x1, x2 = x_offset, x_offset + insect_width - x_reduce_left
    # (3) Length調整
    # 昆虫画像がy方向下側にはみ出る場合は、はみ出る長さをy_reduce_lower変数に格納し、また背景画像合成位置下側(y2)をはみ出る分だけ減算する
    y_reduce_lower = 0
    if (y2 > backimg_height):
        y_reduce_lower = y2 - backimg_height
        y2 -= y_reduce_lower

    # 昆虫画像がx方向右側にはみ出る場合は、はみ出る長さをx_reduce_right変数に格納し、また背景画像合成位置右側(x2)をはみ出る分だけ減算する
    x_reduce_right = 0
    if (x2 > backimg_width):
        x_reduce_right = x2 - backimg_width
        x2 -= x_reduce_right

    #
    # (4) αチャネルプレーンの準備
    # 画像はみ出しを考慮し、矩形サイスを調節する
    #
    if (y_reduce_upper != 0 and x_reduce_left == 0):	# 昆虫画像がY方向上部にはみ出ており、X方向左にはみ出ていないケース
        alpha_insect = img_insect[y_reduce_upper:insect_height, 0:(x2-x1), 3] / 255.0
    elif (y_reduce_upper == 0 and x_reduce_left != 0):	# 昆虫画像がY方向上部にはみ出ておらず、X方向左にはみ出ているケース
        alpha_insect = img_insect[0:(y2-y1), x_reduce_left:insect_width, 3] / 255.0
    else: # 昆虫画像がY方向上部にも、X方向左にもはみ出ていないケース
        
        alpha_insect = img_insect[0:(y2-y1), 0:(x2-x1), 3] / 255.0

    alpha_back = 1.0 - alpha_insect

    #
    # (5) BGRプレーン合成処理
    #
    for c in range(0, 3):
        if (y_reduce_upper != 0 and x_reduce_left == 0):	# 昆虫画像がY方向上部にはみ出ており、X方向左にはみ出ていないケース
            img_1 = alpha_insect * img_insect[y_reduce_upper:insect_height, 0:(x2-x1), c]
        elif (y_reduce_upper == 0 and x_reduce_left != 0):	# 昆虫画像がY方向上部にはみ出ておらず、X方向左にはみ出ているケース
            img_1 = alpha_insect * img_insect[0:(y2-y1), x_reduce_left:insect_height, c]
        else: # 昆虫画像がY方向上部にも、X方向左にもはみ出ていないケース
            img_1 = alpha_insect * img_insect[0:(y2-y1), 0:(x2-x1), c]

        img_2 = alpha_back * img_back[y1:y2, x1:x2, c]
        img_back[y1:y2, x1:x2, c] = img_1 + img_2
        
    # 画像読み込みとアルファチャンネルの確認（RGB or RGBA対応）
    img_insect = cv2.imread('images/line_AI_c/mantis6_AI.png', cv2.IMREAD_UNCHANGED)

    if img_insect is not None:
        if img_insect.shape[2] == 4:  # RGBA画像の場合
            # アルファチャンネルが存在する場合の処理
            alpha_insect = img_insect[0:(y2-y1), 0:(x2-x1), 3] / 255.0
            img_insect_rgb = img_insect[:, :, :3]  # RGBチャンネルのみ抽出
        elif img_insect.shape[2] == 3:  # RGB画像の場合
            # RGB画像の場合、アルファチャンネルを1（不透明）として扱う
            alpha_insect = np.ones((y2-y1, x2-x1), dtype=float)
            img_insect_rgb = img_insect  # そのまま使用
        else:
            raise ValueError("画像形式がサポートされていません。")
    else:
        raise FileNotFoundError("画像が読み込まれていません。ファイルパスを確認してください。")




# ***********************************************************************
# 3 基準線分沿いの昆虫画像配置
#   昆虫画像1個を、与えられた開始点/終了点に適合する位置に配置する
# ***********************************************************************
def place_insect_along_line(back_img, insect_start_point, insect_end_point, idx, draw_figure, disp_insect_window):

    # 2点を結ぶ直線を描画
    if draw_figure == 1:
        cv2.line(back_img, pt1=insect_start_point, pt2=insect_end_point, color=GREEN, thickness=THCK)

    # 楕円を描画 
    mp_x, mp_y = calc_midpoint(insect_start_point, insect_end_point)
    dist = calc_distance(insect_start_point, insect_end_point)
    angle = calc_angle(insect_start_point, insect_end_point)

    if draw_figure == 1:
        cv2.ellipse(back_img, (mp_x, mp_y), (dist//4, dist//2), angle, 0, 360, color=CYAN, thickness=THCK)	# 縦長の楕円

    # 内側に平行移動した楕円を描画################################試
    delta_x = insect_end_point[0] - insect_start_point[0]
    delta_y = insect_end_point[1] - insect_start_point[1]
    
    direction_x = delta_y / dist
    direction_y = (-1) * delta_x / dist

    mp_x_moved = int(mp_x + direction_x * (dist//4))
    mp_y_moved = int(mp_y + direction_y * (dist//4))
#######################################################################    
    mp_x_moved_2 = int(mp_x + direction_x * (dist//4)/1.7)
    mp_y_moved_2 = int(mp_y + direction_y * (dist//4)/1.7)
#####################################################################
    if draw_figure == 1:
        cv2.ellipse(back_img, (mp_x_moved, mp_y_moved), (dist//4, dist//2), angle, 0, 360, color=RED, thickness=THCK)	# 縦長の楕円
        cv2.ellipse(back_img, (mp_x_moved_2, mp_y_moved_2), (dist//4, dist//2), angle, 0, 360, color=YELLOW, thickness=THCK)

##################################################        
    # 昆虫画像のロード
    if USE_LINE_INSECT == 1:
        insect_file_path = './images/line/' + insect_images_all_2[idx % len(insect_images_all_2)] + '_line.png'
    elif USE_LINE_INSECT == 2:
        insect_file_path = './images/line_AI_c/' + insect_images_AI_c[idx % len(insect_images_AI_c)] + '_AI.png'
    elif USE_LINE_INSECT == 3:
        insect_file_path = './images/wakuta_c/' + insect_images_AI_c_new[idx % len(insect_images_AI_c_new)] + '_AI.png'
    else:
        insect_file_path = './images/256/' + insect_images_all_2[idx % len(insect_images_all_2)] + '_256.png'
   
##################################################all

    insect_img = cv2.imread(insect_file_path, cv2.IMREAD_UNCHANGED)

    if disp_insect_window >= 2:
        disp_img('insect_img', insect_img)

    # ===============================================================
    # αチャネル付き昆虫画像をアフィン変換(回転/拡縮の実行)
    # ===============================================================
    insect_height, insect_width = (insect_img.shape[0] , insect_img.shape[1])
    scale = dist / insect_height * 1.5	# 画像内の余白部を考慮し、拡大する

    insect_size = (insect_width, insect_height)
    center = (insect_width / 2, insect_height / 2)

    angle_mod = (-angle + 180 + 360) % 360

    # 回転変換行列の算出
    rot_mtx = cv2.getRotationMatrix2D(center, angle_mod, scale)

    # アフィン変換
    insect_img_rot = cv2.warpAffine(insect_img, rot_mtx, insect_size, flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT)

    if disp_insect_window >= 1:
        disp_img('insect_img_rot', insect_img_rot)

    # 合成処理#############要改善
    process_image(insect_img_rot, back_img, mp_x_moved_2, mp_y_moved_2)
    
# //////////////////////////////////////////////////////
# 1. メイン関数
# //////////////////////////////////////////////////////
def main():

    # ***************************************
    # 1.1 背景画像ロード処理
    # ***************************************

    #中間層の分割数を入力
    #1で輪郭配置
    PLACE_EDGE_INSECTS = int(1)

    # 元画像の読み込み 640×427
    img_back = cv2.imread('images/background/back_wakuta.jpg')
    if DISP_DOG_WINDOW >= 1:
        disp_img('img_back', img_back)

    # 描画用に元画像をコピーする
    copied_img = img_back.copy()
    
    # ********************************************
    # 1.2 周辺部・中央部への昆虫画像配置処理
    # ********************************************

    # 輪郭抽出処理
    img_mask = cv2.imread('images/background/wakuta_mask.png')
    print(img_mask)
    contour_back = get_largest_contour(img_mask, disp_image=(1 if DISP_DOG_WINDOW == 2 else 0))

    #
    # 周辺部への配置処理
    #
    points_outer = []

    # 輪郭近似直線群、交点群取得処理
    points_outer = cv2.approxPolyDP(contour_back, 0.003*cv2.arcLength(contour_back, True), True)

    # ここでは、輪郭疑似直線群の取得までを行う
    # 画像前面/背面の都合により、直線内昆虫画像の描画は、後で行う
    # 輪郭直線毎昆虫配置処理#################################################################
    if PLACE_EDGE_INSECTS == 1:

        draw_fig = (1 if (DRAW_EDGE_FIGURE == 1) else 0)	# ガイド図形描画有無

        for i in range(0, len(points_outer)):
            insect_start_point = [points_outer[i][0][0], points_outer[i][0][1]]
            j = (i+1) if (i+1) != len(points_outer) else 0
            insect_end_point = [points_outer[j][0][0], points_outer[j][0][1]]

            place_insect_along_line(copied_img, insect_start_point, insect_end_point, i, draw_fig, DISP_EDGE_INSECT_WINDOW)

       #犬の顎部分のみ輪郭配置、目安i=9くらい
        ''' for i in range(4, 7):
            insect_start_point = [points_outer[i][0][0], points_outer[i][0][1]]
            j = (i+1) if (i+1) != len(points_outer) else 0
            insect_end_point = [points_outer[j][0][0], points_outer[j][0][1]]

            place_insect_along_line(copied_img, insect_start_point, insect_end_point, i, draw_fig, DISP_EDGE_INSECT_WINDOW)
      '''
    # ***************************************
    # 1.4 合成画像表示・ファイル出力処理
    # ***************************************

    disp_img('copied_img', copied_img)
    cv2.imwrite("./results/results_wakuta/wakuta16_result2.png", copied_img)

if __name__ == "__main__":
    main()


[[[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 ...

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]

 [[0 0 0]
  [0 0 0]
  [0 0 0]
  ...
  [0 0 0]
  [0 0 0]
  [0 0 0]]]
